In [ ]:
# Kimlik Doğrulama.
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


`Hissiyat analizi` metinlerdeki öznel bilgiyi bulmak için bağlamsal bir data mining yöntemidir. Hissiyat analizi üretilen ürünlerin ya da sunulan hizmetin kişiler tarafından beğenilip beğenilmediği ya da bir görüş belirtmek istemediği durumları anlamamıza yardımcı olur.

In [ ]:
# ilgili kütüphane ve veri setinin yüklenmesi.
import pandas as pd
data = pd.read_excel("/gdrive/My Drive/Calisma_ortami/Uygulamalar/reviews.xlsx")
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
data.head()

In [ ]:
data['label'].value_counts()

In [ ]:
# veri ön işleme
import string
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

In [ ]:
noktalama = string.punctuation
etkisiz = stopwords.words('turkish')
print(noktalama)
print(etkisiz)

In [ ]:
for d in data['review'].head():
  print(d+ '\n----------------------')
  # etkisiz kelimelerin atılması
  temp = ''
  for word in d.split():
    if word not in etkisiz and not word.isnumeric():
      temp += word + ' '
  print(temp + '\n********************')

In [ ]:
for d in data['review'].head():
  print(d+ '\n------------------')
  temp = ''
  for word in d:
    if word not in noktalama:
      temp += word
  print(temp + '\n*********************')
  d = temp

In [ ]:
data.to_csv(r'/gdrive/My Drive/Calisma_ortami/Uygulamalar/cleaned.csv', index = False)

In [ ]:
# temizlenmiş veriyi yüklüyoruz.
import pandas as pd
data = pd.read_csv('/gdrive/My Drive/Calisma_ortami/Uygulamalar/cleaned.csv', sep=",")
print(data.head())

                                              review label
0  çok şık ve kaliteli görünümlü bir ürün. banyo ...   pos
1  ürün istanbul içi geç teslimatla 4 günde elime...   neg
2  schneider ürünlerinde problem yaşamadım hiçbir...   pos
3  ürünü saat 11:00 de si̇pari̇ş etti̇m ertesi̇ g...   pos
4  sırt kısmına önem veririm ve büyük olması beni...   pos


In [ ]:
# temizlenmiş veri setini train ve test olarak ayırıyoruz.
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data['review'].head(30000).values.astype('U'),
                                                 data['label'].head(30000).values.astype('U'), test_size=0.33,random_state=42)
print(X_train.shape)
print(X_test.shape)

(20100,)
(9900,)


In [ ]:
# train kümesindeki cümlelerin sayma vektörlerini çıkarıyoruz.
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts.shape)

(20100, 50425)


In [ ]:
# train kümesindeki cümlelerin TF*IDF vektörlerini sayma vektörlerinden oluşturuyoruz.
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tdidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tdidf.shape)

(20100, 50425)


In [ ]:
# çok modlu naive bayes sınıflandırıcı eğitiyoruz.
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tdidf,y_train)
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [ ]:
# sınıflandırıcı ile test seti üzerinden tahminleme yapıyoruz.
y_pred = clf.predict(X_test_tfidf)
for review, sentiment in zip(X_test[:-5],y_pred[:]):
  print('%r => %s' % (review,sentiment))

In [ ]:
# performans sonuçları
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.9393939393939394
